## Machine Learning - Lista 3
### Aluno: Douglas Gaspar Feitosa Freitas
### Matrícula: 473552

## Questão 1

Considere o conjunto de dados disponível em **kc2.csv**, organizado em 22 colunas, sendo as 21 primeiras colunas os atributos e a última coluna a saída. Os 21 atributos são referentes à caracterização de códigos-fontes para processamento de dados na NASA. A saída é a indicação de ausência (0) ou existência (1) de defeitos (os dados foram balanceados via subamostragem). Maiores detalhes sobre os dados podem ser conferidos em *https://www.openml.org/search?type=data&sort=runs&id=1063&status=active*.

a) Considerando uma validação cruzada em 10 folds, avalie modelos de classificação binária nos dados em questão. Para tanto, use as abordagens abaixo:
- **KNN** (escolha k = 1 e k = 5, distância Euclidiana e Mahalonobis, totalizando 4 combinações);
- **Árvore de Decisão** (você pode usar uma implementação já existente, como a do scikit-learn, com índices de impureza de gini e entropia).
 
b) Para cada modelo criado, reporte valor médio e desvio padrão das métricas de **acurácia**, **revocação**, **precisão** e **F1-score**.

In [10]:
import pandas as pd
import numpy as np

# Carregar o dataset kc2.csv
dataset = pd.read_csv('kc2.csv')

# Separar atributos (X) e a variável alvo (y)
X = dataset.iloc[:, :-1].values  # todas as colunas menos a última
y = dataset.iloc[:,-1:].values   # última coluna

# Conferir formatos
print(X.shape)
print(y.shape)


(213, 21)
(213, 1)


### KNN

In [26]:
import numpy as np
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.spatial.distance import cdist

# Função para calcular a distância entre dois pontos (Euclidiana e Mahalanobis)
def calculate_distance(X_train, X_test, metric='euclidean', VI=None):
    if metric == 'euclidean':
        return cdist(X_test, X_train, metric='euclidean')
    elif metric == 'mahalanobis':
        return cdist(X_test, X_train, metric='mahalanobis', VI=VI)
    else:
        raise ValueError("Unsupported metric")

# Implementação do KNN manual
def knn_predict(X_train, y_train, X_test, k=1, metric='euclidean'):
    if metric == 'mahalanobis':
        VI = np.linalg.inv(np.cov(X_train.T))
        distances = calculate_distance(X_train, X_test, metric='mahalanobis', VI=VI)
    else:
        distances = calculate_distance(X_train, X_test, metric='euclidean')
    
    # Para cada ponto de teste, encontrar os k vizinhos mais próximos
    neighbors_idx = np.argsort(distances, axis=1)[:, :k]
    neighbors_labels = y_train[neighbors_idx]
    
    # Prever a classe mais comum entre os vizinhos (corrigido)
    y_pred = [Counter(neighbors.flatten()).most_common(1)[0][0] for neighbors in neighbors_labels]
    return np.array(y_pred)

# Função para rodar a validação cruzada manualmente
def cross_validate_knn(X, y, k=1, metric='euclidean', n_splits=10):
    skf = StratifiedKFold(n_splits=n_splits)
    
    accuracies, precisions, recalls, f1s = [], [], [], []
    
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        y_pred = knn_predict(X_train, y_train, X_test, k=k, metric=metric)
        
        # Calcular as métricas
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
    
    # Retornar as métricas médias e desvio padrão
    return {
        'accuracy_mean': np.mean(accuracies), 'accuracy_std': np.std(accuracies),
        'precision_mean': np.mean(precisions), 'precision_std': np.std(precisions),
        'recall_mean': np.mean(recalls), 'recall_std': np.std(recalls),
        'f1_mean': np.mean(f1s), 'f1_std': np.std(f1s)
    }

# Testando KNN com k=1 e k=5 usando a distância Euclidiana e Mahalanobis
results_knn_manual = {}
for k in [1, 5]:
    for metric in ['euclidean', 'mahalanobis']:
        results = cross_validate_knn(X, y, k=k, metric=metric)
        results_knn_manual[f'KNN_k={k}_{metric}'] = results

# Exibir resultados KNN
for key, value in results_knn_manual.items():
    print(f"{key}:")
    print(f"  Acurácia média = {value['accuracy_mean']:.4f}, Desvio padrão = {value['accuracy_std']:.4f}")
    print(f"  Precisão média = {value['precision_mean']:.4f}, Desvio padrão = {value['precision_std']:.4f}")
    print(f"  Revocação média = {value['recall_mean']:.4f}, Desvio padrão = {value['recall_std']:.4f}")
    print(f"  F1-score média = {value['f1_mean']:.4f}, Desvio padrão = {value['f1_std']:.4f}\n")

KNN_k=1_euclidean:
  Acurácia média = 0.6794, Desvio padrão = 0.1456
  Precisão média = 0.6711, Desvio padrão = 0.1703
  Revocação média = 0.6655, Desvio padrão = 0.2249
  F1-score média = 0.6584, Desvio padrão = 0.2044

KNN_k=1_mahalanobis:
  Acurácia média = 0.6513, Desvio padrão = 0.1644
  Precisão média = 0.6099, Desvio padrão = 0.2387
  Revocação média = 0.6464, Desvio padrão = 0.2615
  F1-score média = 0.6233, Desvio padrão = 0.2417

KNN_k=5_euclidean:
  Acurácia média = 0.7727, Desvio padrão = 0.1872
  Precisão média = 0.7039, Desvio padrão = 0.2770
  Revocação média = 0.7427, Desvio padrão = 0.3447
  F1-score média = 0.7152, Desvio padrão = 0.3098

KNN_k=5_mahalanobis:
  Acurácia média = 0.7305, Desvio padrão = 0.1534
  Precisão média = 0.7724, Desvio padrão = 0.2779
  Revocação média = 0.6018, Desvio padrão = 0.2754
  F1-score média = 0.6555, Desvio padrão = 0.2606



### Árvore de Decisão

In [27]:
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# Função para rodar validação cruzada na árvore de decisão
def cross_validate_decision_tree(X, y, impurity_func, max_depth=5, n_splits=10):
    skf = StratifiedKFold(n_splits=n_splits)
    
    accuracies, precisions, recalls, f1s = [], [], [], []
    
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        tree = build_tree(X_train, y_train, impurity_func, max_depth=max_depth)
        
        y_pred = [predict_tree(tree, x) for x in X_test]
        
        # Calcular as métricas
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, zero_division=0))
        recalls.append(recall_score(y_test, y_pred, zero_division=0))
        f1s.append(f1_score(y_test, y_pred, zero_division=0))
    
    return {
        'accuracy_mean': np.mean(accuracies), 'accuracy_std': np.std(accuracies),
        'precision_mean': np.mean(precisions), 'precision_std': np.std(precisions),
        'recall_mean': np.mean(recalls), 'recall_std': np.std(recalls),
        'f1_mean': np.mean(f1s), 'f1_std': np.std(f1s)
    }

# Testar árvore de decisão com gini e entropia
results_dt = {}
for impurity_func, name in [(gini_impurity, 'gini'), (entropy_impurity, 'entropy')]:
    results = cross_validate_decision_tree(X, y, impurity_func)
    results_dt[f'DecisionTree_{name}'] = results

# Exibir resultados Árvore de Decisão
for key, value in results_dt.items():
    print(f"{key}:")
    print(f"  Acurácia média = {value['accuracy_mean']:.4f}, Desvio padrão = {value['accuracy_std']:.4f}")
    print(f"  Precisão média = {value['precision_mean']:.4f}, Desvio padrão = {value['precision_std']:.4f}")
    print(f"  Revocação média = {value['recall_mean']:.4f}, Desvio padrão = {value['recall_std']:.4f}")
    print(f"  F1-score média = {value['f1_mean']:.4f}, Desvio padrão = {value['f1_std']:.4f}\n")

DecisionTree_gini:
  Acurácia média = 0.7273, Desvio padrão = 0.1047
  Precisão média = 0.7274, Desvio padrão = 0.1424
  Revocação média = 0.6945, Desvio padrão = 0.2354
  F1-score média = 0.6948, Desvio padrão = 0.1904

DecisionTree_entropy:
  Acurácia média = 0.7171, Desvio padrão = 0.1260
  Precisão média = 0.6896, Desvio padrão = 0.2376
  Revocação média = 0.6745, Desvio padrão = 0.2688
  F1-score média = 0.6710, Desvio padrão = 0.2377

